# TODOs
- [ ] Runtimes with various input size and different optimization flags
> Create a timing infrastructure to get runtimes for a suitable set of input sizes (and inputs). Push sizes to the limit (i.e., until the execution takes minutes or even hours). Consider different compiler flags (but always with optimization).
- [ ] Cost Analysis (float ops, float comp, int ops, int comp)
> Cost analysis: Select a cost measure (usually flop count but may include div, sin, ...) and determine the cost. Briefly say how you did it (e.g., counted in code, had to instrument parts, ...).
- [ ] Performance Plot
> Create a performance plot. Put it as plot0 (or plot0a, plot0b, ...) into svn in the same directory as this document. As you perform optimizations later, include one line for each relevant optimization.
- [ ] Bottlenecks by profiling tool
> Identify performance bottlenecks in your code by profiling. This will likely involve instrumenting your code (standard profilers may not be fine grain enough). Focus on those parts when optimizing. The bottlenecks may change during optimization. Understand the cost and performance of the bottlenecks to set expectations right.

In [1]:
import re
import numpy as np    
import pandas as pd

general_log = "./general_metrics.txt"
cache_log = './cache_log.txt'
flops_log = './flops_log.txt'

In [2]:
def extract(log, metric):
    res = []
    with open(log, 'r') as f:
        file = f.read()
    try:
        if metric=='runtime(sec)':
            res = re.findall("(.*) seconds time elapsed", file)
        elif metric=='cc_int_ops':
            res = re.findall("cc_int_ops:(.*), cc_int_comp", file)
        elif metric=='cc_int_comp':
            res = re.findall("cc_int_comp:(.*), cc_fl_ops", file)
        elif metric=='cc_fl_ops':
            res = re.findall("cc_fl_ops(.*), cc_fl_comp", file)
        elif metric=='cc_fl_comp':
            res = re.findall("cc_fl_comp:(.*)", file)
        elif metric=='ll_int_ops':
            res = re.findall("ll_int_ops:(.*), ll_int_comp:", file)
        elif metric=='ll_int_comp':
            res = re.findall("ll_int_comp:(.*)", file)
        elif metric=='flops':
            res = re.findall("#fl ops: (.*)", file)
        elif metric=='flcomp':
            res = re.findall("#fl comparisons: (.*)", file)
        elif metric=='intops':
            res = re.findall("#int ops: (.*)", file)
        elif metric=='intcomp':
            res = re.findall("#int comparisons: (.*)", file)
        elif metric=='cycles':
            res = re.findall("#cycles: (.*)", file)
        elif metric=='flops/cycles':
            res = re.findall("(.*)flops / cycles", file)
        elif metric=='instructions':
            res = re.findall("(.*)   instructions:u", file)
        elif metric=='branches':
            res = re.findall("(.*)     branches:u", file)
        elif metric=='branch-misses':
            res = re.findall("branch-misses:u           #(.*) of all branches", file)
        elif metric=='page-faults':
            res = re.findall("(.*)  page-faults:u", file)
        elif metric=='cycles(perf)':
            res = re.findall("(.*)      cycles:u", file)
        elif metric=='flops(perf)':
            res = re.findall("(.*) r5301c7", file)
        elif metric=='L1-dcache-loads':
            res = re.findall("(.*)L1-dcache-loads:u", file)
        elif metric=='L1-dcache-load-misses':
            res = re.findall("L1-dcache-load-misses:u   #(.*)\% ", file)
        elif metric=='LLC-loads':
            res = re.findall("(.*)LLC-loads:u", file)
        elif metric=='LLC-load-misses':
            res = re.findall("LLC-load-misses:u         #(.*)\% ", file)
    except Exception:
        return 
    return res  

def init_table(vertice, edge):
    ve = []
    for v in vertice:
        for e in edge:
            if e < v*10 and e >= v:
                ve.append([v,e])
    return pd.DataFrame(ve, columns=['#V', '#E'])

def update_table(df, log, data, header):
    for i, x in enumerate(df['#V']):
        for j, y in enumerate(df['#E']):
            for l in header:
                data[l] = extract(log, l)
                data[l] = [float(x.replace(',', '').strip().strip('%')) for x in data[l]]
                if len(data[l])==len(df):
                    df[l] = data[l]
                else:
                    print(f"fail to add {l} to the table.")

In [3]:
from itertools import product

label=['runtime(sec)',
       'flops','flcomp','intops','intcomp', 'flops/cycles',
      #'cc_int_ops', 'cc_int_comp', 'cc_fl_ops', 'cc_fl_comp', 'll_int_ops', 'll_int_comp',
      'instructions', 'cycles', 'cycles(perf)', 
      'branches', 'branch-misses', 'page-faults']
cache=['L1-dcache-loads','L1-dcache-load-misses','LLC-loads','LLC-load-misses',
      # 'LLC-stores','LLC-store-misses','L1-dcache-stores', 'dTLB-loads','dTLB-load-misses','dTLB-stores','dTLB-store-misses',
      # 'iTLB-loads', 'iTLB-load-misses'
       ]
flops=['flops(perf)']

vertice = [10,100,1000,2000,5000,8000,10000]
edge = [10,100,500,1000,2000,5000,8000,10000,20000,50000]

In [4]:
# results with flag -O3
data = dict()
res = init_table(vertice, edge)
res = res[:-1]
update_table(res, general_log, data, label)
update_table(res, flops_log, data, cache)
#res.to_excel('result.xlsx', sheet_name='sheet1', index=False) 
res.convert_dtypes()
res.dtypes

#V                         int64
#E                         int64
runtime(sec)             float64
flops                    float64
flcomp                   float64
intops                   float64
intcomp                  float64
flops/cycles             float64
instructions             float64
cycles                   float64
cycles(perf)             float64
branches                 float64
branch-misses            float64
page-faults              float64
L1-dcache-loads          float64
L1-dcache-load-misses    float64
LLC-loads                float64
LLC-load-misses          float64
dtype: object

In [5]:
import plotly.express as px
import plotly.graph_objects as go

num_font = dict(size=22,color='#000000')
default_font = dict(family="Helvetica Neue",size=20,color='#000000')
#colors=px.colors.sequential.deep[2:]  # Candidate: Prism
colors=px.colors.sequential.Cividis[::-1][1:]
marker_symbols=['circle','square','diamond','cross','triangle-up-open','pentagon','hexagram','star','hourglass', 'asterisk']

In [6]:
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["flops/cycles"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_flops.pdf")

In [7]:
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["flcomp"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_flcomp.pdf")

In [8]:
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["cycles"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_cycles.pdf") #similar graph for runtime, 

In [9]:
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["page-faults"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_page-faults.pdf")

In [10]:
# branch-misses
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["branch-misses"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_branch_miss.pdf")

In [11]:
# runtime
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["runtime(sec)"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_runtime.pdf") # similar graph for cycles, 

In [12]:
# L1-dcache-load-misses	
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["L1-dcache-load-misses"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_L1-dcache-load-misses.pdf")

In [13]:
# LLC-load-misses
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["LLC-load-misses"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_LLC-load-misses.pdf")

In [14]:
# LLC-load vs L1-dcache-loads
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["L1-dcache-loads"]/same_e["LLC-loads"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_L1toLLC.pdf")

In [15]:
# operation intensity: instruction/memory
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    man_instr = same_e["flops"]+same_e["flcomp"]+same_e["intops"]+same_e["intcomp"]
    operation = same_e["instructions"]/same_e["L1-dcache-loads"]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=operation, name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_oi.pdf")

In [16]:
# flops/cycle vs oi
fig = go.Figure()

fig.add_trace(go.Scatter(x=res["flops"]/res["L1-dcache-loads"], y=res["flops/cycles"], 
                         mode='lines+markers',marker=dict(symbol=marker_symbols[3],size=5),
                         line=dict(color=colors[3], width=1)))

fig.update_layout(xaxis=dict(showline=True, showgrid=True,
                             linecolor='rgb(0, 0, 0)', linewidth=2,rangemode="nonnegative",
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,
                             rangemode="nonnegative",
                             showline=True),
                  #showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_roofline.pdf")